# Práctica 2

Gonzalo Monedero - Marcos Molina - Nicolás Salgado - Iñigo Vázquez

## Preparación del entorno

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import keras as ker
import tensorflow as tf
import random
import time
import collections
import statistics

from statistics import mode
from sklearn.preprocessing import MinMaxScaler

# Quitar warnings feos
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

## Preprocesamiento de datos

### Creación del dataset


In [2]:
#leemos los datos aqui
iris_RAW = pd.read_csv('../Data/iris.data', sep=',')
iris_OG = iris_RAW.copy()
salidas = iris_RAW.pop("class")

## Creación entradas y salidas

In [3]:
entradas = iris_RAW.to_numpy()
mm = MinMaxScaler()
entradas = mm.fit_transform(entradas)

salidas = pd.get_dummies(salidas)

# Funciones algoritmo genético

### Variables a definir

In [4]:
"""
ngenes: nneuronas de cada individuo. Sacado a traves del L4P1-EstudioRed.ipynb
NPOB: Número de individuos de la población 
NGEN: Número de generaciones (ciclo completo sobre todos los individuos) 
CONVGEN: Número de generaciones que si no hay cambio para.
Pm: Probabilidad de mutación: Probabilidad de que un gen dado mute 
NRES: Cada cuántas generaciones se imprime un resumen de la evolución del proceso 
NSAMPLE: Cada qué número de generaciones se saca un muestreo de la población 
elitismo: Porcentaje de elitismo, es decir, porcentaje de mejores individuos guays que generan 4 hijos
"""

ngenes = 6 # NO TOCAR
npob = 50 # NO TOCAR
ngeneraciones = 100 # TOCAR
convgen = 10 # TOCAR
pm = 0.05 # TOCAR
nres = 10 # TOCAR
nsample = 30 # TOCAR
elitismo = 0.2 # TOCAR

### Crear inidividuo

Como bien se dice en el enunciado, un individuo es un MLP sin entrenar, con tantos genes como hemos calculado. En nuestro caso son 7 genes, y el tamaño de entrada son 4. De esta manera creamos un MLP básico.

In [5]:
def crearMLP(ngenesMLP, tam_entrada,tipo,pesos):

    MLP = ker.Sequential() # Creamos el modelo vacío

    # Añadimos la primera capa (tecnicamente dos, la de entrada (con la variable input_dim) y la oculta)
    MLP.add(ker.layers.Dense(ngenesMLP, input_dim = tam_entrada, activation='relu'))

    # Añadimos la capa de salida
    MLP.add(ker.layers.Dense(3,activation='sigmoid')) 

    # Por último tenemos que configurar el modelo antes de entrenarlo con lo dicho en el enunciado
    # Optimizador: Adam
    # Función error: Mean Squared Error
    # Metricas = Binary accuracy porque nos da el número de precisión de la red
    MLP.compile(optimizer='adam',loss='mean_squared_error',metrics=["accuracy"])

    if(tipo == 1): # Le metemos pesos ya predeterminados
        MLP.set_weights(pesos)

    return MLP

### Crear población

Al igual que antes hemos creado un individuo en este caso crearemos una población. Una población es un conjunto de individuos, en nuestro caso entonces realizaremos un array de población

In [6]:
def crearPoblacion(tamPoblacion,ngenesMLP, tam_entrada):
    poblacion = []
    for i in range (tamPoblacion):
        time.sleep(1)
        individuo = crearMLP(ngenesMLP,tam_entrada,0,0)
        poblacion.append(individuo)
    return poblacion

### Evaluar población

El calculo del fitness de la población se realiza mediante el evaluate del MLP. De esta manera, simplemente tendremos que iterar toda la población y devolver un array con el fitness de cada individuo en la posición de la población.

In [7]:
def evaluarPoblacion(poblacion,entradas_test,esperadas_test):
    evalPobl = []
    for individuo in poblacion:
        metricas_totales = round(individuo.evaluate(entradas_test, esperadas_test,verbose=0)[1],3)
        evalPobl.append(metricas_totales*150)
    return evalPobl

### Evaluar individuo

Adicionalmente por si es requerido, crearemos una evaluacion para un unico individuo

In [8]:
def evaluarIndividuo(individuo,entradas_test,esperadas_test):
    metricas_totales = round(individuo.evaluate(entradas_test, esperadas_test,verbose=0)[1],3)
    evalInd = metricas_totales*150
    return evalInd

### Selección de replicar individuo

Posteriormente necesitaremos un algoritmo de montecarlo, que nos permita elegir la probablididad de selección de el individuo. El método de montecarlo es generar un número aleatorio entre [0,1], si este número es menor que la probabilidad de selección, el individuo se replica, si no, no. En nuestro caso, recibimos una población y la recorremos, viendo cuales se cogen o no. De esta manera, devolvemos un de booleano donde indicamos si ese individuo se replica o no.

In [9]:
def seleccionados(poblacion,entradas_test,esperadas_test, elitismo):
    fis_totales = evaluarPoblacion(poblacion,entradas_test,esperadas_test)
    pob_fis = dict(zip(poblacion,fis_totales))

    pob_fis_ordenado = {k: v for k, v in sorted(pob_fis.items(), key=lambda item: item[1], reverse=True)}
    nElegidos = int(npob * elitismo) # X % pobabilidad dependiendo del valor elitismo
    mejores = []
    for individuo in pob_fis_ordenado:
        mejores.append(individuo)
        nElegidos -= 1
        if(nElegidos == 0):
            break
    return mejores

### Mutar

Secuencialmente para todos los genes del padre decide si cada gen se copia fielmente en el hijo o se muta usando un Método de Montecarlo simple sobre el valor de Pm. Si muta, cámbialo por otro valor aleatorio en el rango (-1, 1). Prueba también otras opciones de mutación. 

In [10]:
def mutar(individuo):
    pesos_antiguos = individuo.get_weights()
    pesos_entrada_oculta = pesos_antiguos[0]
    pesos_oculta_salida = pesos_antiguos[2]

    for i in range(len(pesos_entrada_oculta)):
        for j in range(len(pesos_entrada_oculta[i])):
            nAl = random.uniform(0, 1)
            if(nAl < pm):
                pesos_entrada_oculta[i][j] = random.uniform(-1, 1)

    for i in range(len(pesos_oculta_salida)):
        for j in range(len(pesos_oculta_salida[i])):
            nAl = random.uniform(0, 1)
            if(nAl < pm):
                pesos_oculta_salida[i][j] = random.uniform(-1, 1)
    
    aux1 = np.zeros(ngenes, dtype='float32')
    aux2 = np.array([0.,0.,0.], dtype='float32')
    pesos_nuevos = [pesos_entrada_oculta,aux1,pesos_oculta_salida,aux2] # Los pesos con mutación

    nuevoMLP = crearMLP(ngenes,4,1,pesos_nuevos)

    return nuevoMLP

### Cromosoma consenso

El cromosoma consenso es aquel individuo que tiene los genes más repetidos. De esta manera de cada generación podríamos saber un individuo medio, teniendo las cualidades más típicas de la generación

In [11]:
def cromosomaconsenso(poblacion, ngenes):
    matriz1_cro = np.empty((4, ngenes))
    matriz2_cro = np.empty((ngenes, 3))

    for i in range(len(matriz1_cro)):
        for j in range(len(matriz1_cro[i])):
            lista_pos = []
            for individuo in poblacion:
                pesos_antiguos_individuo = individuo.get_weights()
                pesos_ind = pesos_antiguos_individuo[0]
                lista_pos.append(pesos_ind[i][j])
            matriz1_cro[i][j] = mode(lista_pos)
            print(lista_pos)
            print(matriz1_cro[i][j])

    for i in range(len(matriz2_cro)):
        for j in range(len(matriz2_cro[i])):
            lista_pos = []
            for individuo in poblacion:
                pesos_antiguos_individuo = individuo.get_weights()
                pesos_ind = pesos_antiguos_individuo[2]
                lista_pos.append(pesos_ind[i][j])
            matriz2_cro[i][j] = mode(lista_pos)
    
    aux1 = np.zeros(ngenes, dtype='float32')
    aux2 = np.array([0.,0.,0.], dtype='float32')
    pesos_nuevos = [matriz1_cro,aux1,matriz2_cro,aux2] # Los pesos con mutación

    cromosomaCom = crearMLP(ngenes,4,1,pesos_nuevos)

    return cromosomaCom

### Resumen

Cada NRES generaciones escribir un resumen que contenga:

    • Nº de generación 

    • El mejor cromosoma, su fitness (BESTf) y cuantos hay en número (BESTn) y en % (BEST%) 

    • El cromosoma  consenso (aquel obtenido con el peso más frecuente en cada posición) y su fitness (CONSf) 

In [12]:
def resumen(poblacion,entradas,salidas,generacion):
    pobFit = evaluarPoblacion(poblacion,entradas,salidas)
    valMejInd = np.amax(pobFit)
    nInd = collections.Counter(pobFit)[valMejInd]
    cromosomaC = cromosomaconsenso(poblacion,ngenes)
    print("======================= RESUMEN =======================")
    print("Nº generacion", str(generacion+1))
    print("Mejor individuo tiene: ", str(valMejInd))
    print("Hay ", str(nInd), "con el mejor fitness, lo cual es un ", str((nInd/npob)*100),"%")
    print("El cromosoma consenso tiene de fitness ", evaluarIndividuo(cromosomaC,entradas,salidas))
    print("=======================================================")    

### Muestreo

Cada NSAMPLE generaciones escribir

    • El resumen NRES 

    • Un muestreo del 20% de los individuos de la población 

In [13]:
def muestreo(poblacion, entradas, salidas):
    contador = 0
    pobFit = evaluarPoblacion(poblacion, entradas, salidas)
    valMejInd = np.amax(pobFit)
    nInd = collections.Counter(pobFit)[valMejInd]

    nindmuestreo = int(npob * 0.2)  # 20%
    muestreo = random.sample(poblacion, nindmuestreo)
    print("======================= MUESTREO =======================")
    print("Muestro de ", str(nindmuestreo), "individuos")
    for individuoMuestreo in muestreo:
        contador += 1
        print("Individuo ", str(contador), "tiene fitness de", str(
        evaluarIndividuo(individuoMuestreo, entradas, salidas)))
    print("==================================================================")


### Simulación

Simulamos una población durante muchas generaciones. Pese a que la evaluación sea durante muchas generaciones existe una condición de parada basada en que si durante x número de generaciones el mejor fitness no cambia, se para el programa.

In [14]:
def simular():

    print("======================= SIMULACIÓN INICIADA =======================")
    print("Matriz pesos de tamaño: 4 *", str(ngenes) , "y de 3 *", str(ngenes))
    print("NPOB: ", npob)
    print("NGEN: ", ngenes)
    print("Probabilidad mutacion ", pm)
    print("==================================================================")
    
    poblacion = crearPoblacion(npob,ngenes, 4) # Población inicial

    ultimo = 0
    contador = 0
    actual = 0

    for i in range(ngeneraciones):
        nueva_poblacion = []
        print("Generacion ", str(i+1), end="\r")
        
        mejoresIndividuos = seleccionados(poblacion,entradas,salidas,elitismo)

        for indvidio_mutar in mejoresIndividuos:
            for x in range(int(1/elitismo)):
                nuevoIndividuo = mutar(indvidio_mutar) # Lo mutamos
                nueva_poblacion.append(nuevoIndividuo)
        
        if((i+1) % nres  == 0): # imprime un resumen de la evolución del proceso 
            resumen(nueva_poblacion,entradas,salidas,i)
                
        if((i+1) % nsample  == 0): # muestreo de la población 
            muestreo(nueva_poblacion,entradas,salidas)

        poblacion = nueva_poblacion.copy()

        # Condición de parada
        
        actual = np.amax(evaluarPoblacion(poblacion,entradas,salidas))
        if(actual == ultimo):
            contador += 1
            if(contador == convgen):
                print("PARADA, TRAS", str(convgen), "GENERACIONES NO HA HABIDO NINGÚN CAMBIO DEL MEJOR FITNESS ", str(actual))
                return
        else:
            ultimo = actual
            contador = 0

simular()

======================= SIMULACIÓN INICIADA =======================
Matriz pesos de tamaño: 4 * 6 y de 3 * 6
NPOB:  50
NGEN:  6
Probabilidad mutacion  0.05
[-0.13625339, -0.13625339, -0.13625339, -0.13625339, -0.13625339, -0.13625339, -0.13625339, -0.13625339, -0.13625339, -0.13625339, 0.17169297, 0.17169297, 0.17169297, 0.17169297, 0.17169297, 0.17169297, 0.17169297, 0.17169297, 0.17169297, 0.17169297, 0.17169297, 0.17169297, 0.17169297, 0.17169297, 0.17169297, 0.17169297, 0.17169297, 0.17169297, 0.17169297, 0.17169297, 0.17169297, 0.17169297, 0.17169297, 0.17169297, 0.17169297, 0.17169297, 0.17169297, 0.17169297, 0.17169297, 0.17169297, 0.17169297, 0.17169297, 0.17169297, 0.17169297, 0.17169297, 0.17169297, 0.17169297, 0.17169297, 0.17169297, 0.17169297]
0.17169296741485596
[-0.06007582, -0.06007582, -0.06007582, -0.06007582, -0.06007582, -0.06007582, -0.06007582, -0.06007582, -0.06007582, -0.06007582, -0.06007582, -0.06007582, -0.06007582, -0.06007582, -0.06007582, -0.06007582, -0.0